In [40]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.models import Sequential
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
import numpy as np
import cv2

In [7]:
train_df = pd.read_csv("fer2013.csv")
test_df = pd.read_csv("test.csv")

In [8]:
X_train = train_df[train_df["Usage"] == "Training"]
y_train = X_train.drop(["pixels", "Usage"], axis=1)
X_test = train_df[train_df["Usage"] == "PublicTest"]
y_test = X_test.drop(["pixels", "Usage"], axis=1)

In [9]:
X_train = X_train.drop(["Usage", "emotion"], axis=1)
X_test = X_test.drop(["Usage", "emotion"], axis=1)

In [10]:
my_list = X_train["pixels"].to_list()
my_list_1 = X_test["pixels"].to_list()
y_train = y_train["emotion"].to_numpy()
y_test = y_test["emotion"].to_numpy()

In [11]:
width, height = 48, 48
input_shape = (48, 48, 1)

In [12]:
def convert_to_np(my_list):
    faces = []
    for i in my_list:
        face = np.asarray(i.split(" "), dtype=np.uint8).reshape(width, height)
        face = cv2.resize(face.astype('uint8'), (width, height))
        faces.append(face.astype('float32'))
        
    faces = np.asarray(faces)
    faces = np.expand_dims(faces, -1)
    
    return faces

In [13]:
X_train = convert_to_np(my_list)
X_test = convert_to_np(my_list_1)
#y_train = np.asarray(y_train)
#y_test = np.asarray(y_test)

In [14]:
X_train /= 255.
X_test /= 255.

In [15]:
y_train

array([0, 0, 2, ..., 4, 0, 4], dtype=int64)

In [16]:
CLASSES = 7

In [17]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation="relu", input_shape=input_shape))
model.add(MaxPooling2D())
model.add(Conv2D(32, (3, 3), activation="relu"))
model.add(Flatten())
model.add(Dense(64, activation="relu"))
model.add(Dense(CLASSES, activation='softmax'))

In [18]:
model.compile(loss='sparse_categorical_crossentropy',
                            optimizer='adam', metrics=['accuracy'])

In [20]:
model.fit(X_train, y_train,
                        validation_data=(X_test, y_test),
                        epochs=5, batch_size=32, verbose=1)

Epoch 1/5
898/898 [==============================] - 8s 4ms/step - loss: 1.5919 - accuracy: 0.3796 - val_loss: 1.4818 - val_accuracy: 0.4299
Epoch 2/5
898/898 [==============================] - 4s 4ms/step - loss: 1.3666 - accuracy: 0.4774 - val_loss: 1.3660 - val_accuracy: 0.4645
Epoch 3/5
898/898 [==============================] - 4s 4ms/step - loss: 1.2400 - accuracy: 0.5298 - val_loss: 1.3193 - val_accuracy: 0.4976
Epoch 4/5
898/898 [==============================] - 4s 4ms/step - loss: 1.1219 - accuracy: 0.5799 - val_loss: 1.3147 - val_accuracy: 0.5077
Epoch 5/5
898/898 [==============================] - 4s 4ms/step - loss: 1.0111 - accuracy: 0.6241 - val_loss: 1.3229 - val_accuracy: 0.5124


In [22]:
model.predict(X_test)

array([[2.6218829e-01, 2.4099855e-03, 1.4945986e-02, ..., 5.0926894e-01,
        2.2735114e-05, 1.9904514e-01],
       [2.8565919e-01, 5.1433410e-02, 1.1490626e-01, ..., 1.2711045e-01,
        3.4178872e-02, 1.7962347e-01],
       [2.6084310e-01, 2.1389227e-02, 4.3011758e-02, ..., 5.0717467e-01,
        2.2403442e-03, 1.6457321e-01],
       ...,
       [7.4908067e-03, 7.2855291e-05, 6.6949002e-02, ..., 5.8274293e-01,
        1.3847837e-02, 8.3207943e-02],
       [6.4134234e-01, 5.5687106e-03, 6.7332357e-02, ..., 1.4162666e-01,
        2.0890613e-03, 1.3623399e-01],
       [4.7673691e-02, 1.4910622e-03, 1.2993024e-01, ..., 3.1837517e-01,
        4.9539786e-02, 4.5135340e-01]], dtype=float32)

In [23]:
model.save("emotion.h5")

In [38]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\Waryk\AppData\Local\Temp\tmpugdz8bzb\assets
